In [1]:
import pandas as pd
import random
from itertools import permutations

# pandas format - 1 decimal floats
pd.set_option('display.float_format', '{:.1f}'.format)
pd.set_option('display.max_columns', None)

### Emojis 😵‍

In [2]:
"""
extra emojis!!!
(watch out for odd extra hidden characters, will add silly columns to your dataframe)
👺⚡🎯🛡️⭐🐍🔴🔵
🗡️⚔🪓🛡️💥🔥🟥🟦🆘
🧌💀💩👾👹😈👿👽
⏸️☣️🦂🤢☠️💫✨➕👘👕🎽
🤺🥷🏼🧙‍♂️😵💤🛌🔮🐉☆🧪🕸🕸️⚠️❌🪖🧱🕷
⏸😵‍💫⛊🔰⛨🛡️🩸💔🫷📏🎲
"""

# rolls
red = '🟥'
blue = '🟦'
hit = '💥'
shield = '⛊'
lightning = '⚡'
SS = '🐉'
star = '☆'
goblin = '😈'

# effects
armour = '🧱'
KO = '🛌'
stun = '⏸'
poison = '🤢'
critical = '❌'
lethal = '💀'
slow = '🕸'
bleed = '🩸'
bash = '⛨'
brek = '💔'  # misspelled to avoid breaking 'break'
AoE = '📏'
attack = '⚔'
reroll = '🎲'
fire = '🔥'
suffer = '😵'
shield_buff = '⛊'
shield_debuff = '🛡'

### Classes and Functions

In [3]:
class Red:
    POSSIBLE_RESULTS = [
        [SS],
        [hit, hit],
        [lightning, lightning],
        [hit, lightning],
        [hit, lightning],
        [hit, lightning],
        [hit, star],
        [hit, star],
        [hit],
        [goblin]
    ]
    
    def __init__(self):
        self.colour = red
    
    def roll(self):
        return [self.colour] + random.choice(self.POSSIBLE_RESULTS)

    
class Blue(Red):
    POSSIBLE_RESULTS = [
        [shield],
        [shield],
        [shield],
        [shield],
        [hit, lightning],
        [hit, lightning],
        [hit, star],
        [hit, star],
        [hit, star],
        [goblin]
    ]
    
    def __init__(self):
        self.colour = blue

    
class Dice:
    def __init__(self, n_red=0, n_blue=0):
        self.n_red = n_red
        self.n_blue = n_blue
        self._dice = [Red() for _ in range(n_red)] + [Blue() for _ in range(n_blue)]
        self.current_roll = None
    
    def roll(self):
        result = []
        for die in self._dice:
            die_result = die.roll()
            result.append(die_result)
        self.current_roll = result
        return self.current_roll
    
    def reroll(self, n_to_reroll):
        """zero indexed - 1st item in n_to_reroll=0"""
        reroll = self._dice[n_to_reroll].roll()
        self.current_roll[n_to_reroll] = reroll
        return self.current_roll
    
    
class Item:
    def __init__(self, n_red=0, n_blue=0, modifiers=None, effects=None, thorgren=False):
        self.dice = Dice(n_red, n_blue)
        self.modifiers = modifiers
        self.effects = effects
        self.current_roll = None
        self.current_totals = None
        self.thorgren = thorgren
        self.roll()
        
    def roll(self, focus=False):
        if self.dice:
            result = self.dice.roll()
        else:
            result = []
        modifier_list = ['modifiers']
        for k,v in self.modifiers.items():
            for _ in range(v):
                modifier_list.append(k)
        result.append(modifier_list)
        if focus:
            result.append(['focus', hit])
        self.current_roll = result    
        self.calc_totals()
        return self.current_roll
    
    def reroll(self, n_to_reroll):
        if self.current_roll[n_to_reroll][0] == 'modifiers':
            print('cannot reroll modifiers')
            return self.current_roll
        self.current_roll[n_to_reroll] = self.dice.reroll(n_to_reroll)[n_to_reroll]
        self.calc_totals()
        return self.current_roll
    
    def print_dice(self):
        if self.current_roll:
            for die in self.current_roll:
                print(die)
            print()
            
    def calc_totals(self):
        if self.current_roll:
            final_results = dict()
            for roll in self.current_roll:
                for result in roll:
                    if result != red and result != blue and result != 'modifiers' and result != 'focus':
                        final_results[result] = final_results.get(result, 0) + 1
        self.current_totals = final_results

    def print_totals(self):
        if self.current_totals == None:
            self.calc_totals()
        for result in self.current_totals.items():
            print(result)
        print()


class Weapon(Item):                      
    def results(self):
        """
        check all effects and list all potential outcomes, incl no effects
        there is definitely a way to make this more efficient, likely needs recursive search algo, leave for now
        """        
        # add an extra effect if self.thorgren == True
        effects = self.effects + [(lightning, hit)] if self.thorgren else self.effects
        
        fx = all_permutations(effects)
        results = []
        for r in self.current_roll:
            results += [x for x in r[1:]]
        results = ''.join(sorted(results, reverse=True))
        columns = set(''.join(sorted([x[1] for x in effects] + [x for x in self.modifiers])))
        columns.discard(hit)
        columns = [hit] + list(columns)  # always show hits first
        output = pd.DataFrame(columns=columns, dtype='int64')

        for f in fx:
            current_result = results 
            if len(f) > 0:
                for cost, effect in f:
                    current_result = current_result.replace(cost, effect, 1)
            counts = {col: current_result.count(col) for col in output.columns}
            output = pd.concat([output, pd.DataFrame([counts])])
        
        output = output.drop_duplicates().sort_values(hit, ascending=False).reset_index(drop=True)
        return output
            
            
# calculate permutations of all lengths
def all_permutations(lst):
    for i in range(len(lst)+1):
        for p in permutations(lst, r=i):
            yield p

# simulation
def simulation(obj, n_trials=100, focus=False):
    obj.roll(focus)
    results = pd.DataFrame(obj.results().max()).T
    for _ in range(n_trials-1):
        obj.roll(focus)
        results = pd.concat([results, pd.DataFrame(obj.results().max()).T])
    return results

def summarise_simulation(dataframe, percentiles=[0.01, 0.1, 0.25, 0.5, 0.75, 0.9, 0.99]):
    cols = ['mean', 'std']+[str(int(x*100))+'%' for x in percentiles]+['max']
    output = dataframe.describe(percentiles=percentiles).T
    return pd.concat([output.iloc[:, 1:3], output.iloc[:, 3:].astype(int)], axis=1)


### Items

In [4]:
weapons = {
    'spear': {
        'n_red': 2,
        'n_blue': 2,
        'modifiers': {hit: 1},
        'effects': [(lightning, armour), (lightning+lightning, bleed)],
        'thorgren': True,
    },
    'trident_of_the_deep': {
        'n_red': 2,
        'n_blue': 4,
        'modifiers': {hit: 2},
        'effects': [(lightning+lightning, bash+brek)],
        'thorgren': True,
    },
    'trident_of_the_deep_1h': {
        'n_red': 2,
        'n_blue': 4,
        'modifiers': {hit: 1},
        'effects': [(lightning+lightning, bash+brek)],
        'thorgren': True,
    },
    'dancing_scimitar': {
        'n_red': 2,
        'n_blue': 1,
        'modifiers': {hit: 1},
        'effects': [(lightning, hit+lethal), (star, suffer)],  # lethal 1
        'thorgren': True,
    },
    'dwarven_pickaxe': {
        'n_red': 2,
        'n_blue': 2,
        'modifiers': {hit: 1},
        'effects': [],
        'thorgren': True,
    },
    'dwarven_pickaxe_minions': {
        'n_red': 2,
        'n_blue': 2,
        'modifiers': {hit: 1},
        'effects': [(lightning, hit+hit+hit)],
        'thorgren': True,
    },
    'crystal_wand': {
        'n_red': 1,
        'n_blue': 4,
        'modifiers': {hit: 3, goblin: 0},
        'effects': [(star, lethal)],  # lethal vs demons only
    },
    'cloak_of_the_kings': {
        'n_red': 0,
        'n_blue': 1,
        'modifiers': {hit: 1},
        'effects': [],  # effects are relative to other items
    },
    'auryns_sceptre_0': {
        'n_red': 4,
        'n_blue': 0,
        'modifiers': {},
        'effects': [(lightning+star, slow+bash+KO), (SS, AoE+AoE)],
    },
    'auryns_sceptre_0_cloak': {
        'n_red': 4,
        'n_blue': 0,
        'modifiers': {},
        'effects': [(lightning+star, slow+bash+KO), (SS, AoE+AoE), (star, shield_buff)],
    },
    'auryns_sceptre_1': {
        'n_red': 3,
        'n_blue': 0,
        'modifiers': {},
        'effects': [(lightning+star, slow+bash+KO), (SS, AoE+AoE)],
    },
    'auryns_sceptre_2': {
        'n_red': 2,
        'n_blue': 0,
        'modifiers': {},
        'effects': [(lightning+star, slow+bash+KO), (SS, AoE+AoE)],
    },
    'auryns_sceptre_3': {
        'n_red': 1,
        'n_blue': 0,
        'modifiers': {},
        'effects': [(lightning+star, slow+bash+KO), (SS, AoE+AoE)],
    },
    'wand_of_darts': {
        'n_red': 0,
        'n_blue': 2,
        'modifiers': {},
        'effects': [(lightning, slow), (star, lethal)],
    },
    'thunder_strike': {
        'n_red': 2,  # this changes based on level. fix it?
        'n_blue': 1,
        'modifiers': {},
        'effects': [(lightning, KO), (lightning+star, AoE)],
    },
    'illusionary_attack': {  # changes based on level. fix it?
        'n_red': 1,
        'n_blue': 2,
        'modifiers': {hit: 1},
        'effects': [(lightning, armour+armour), (star, slow)],
    },
    'sceptre_0': {  # number of spaces from enemy, n_blue=4 minus distance to monster
        'n_red': 0,
        'n_blue': 4,
        'modifiers': {hit: 1},
        'effects': [(star, KO), (star+star, stun)],
    },
    'sceptre_1': {
        'n_red': 0,
        'n_blue': 3,
        'modifiers': {hit: 1},
        'effects': [(star, KO), (star+star, stun)],
    },
    'sceptre_2': {
        'n_red': 0,
        'n_blue': 2,
        'modifiers': {hit: 1},
        'effects': [(star, KO), (star+star, stun)],
    },
    'sceptre_3': {
        'n_red': 0,
        'n_blue': 1,
        'modifiers': {hit: 1},
        'effects': [(star, KO), (star+star, stun)],
    },
    'silver_star': {
        'n_red': 2,
        'n_blue': 4,
        'modifiers': {},
        'effects': [(lightning+lightning, attack), (star, reroll), (SS, stun)],
    },
    'crystal_hammer': {
        'n_red': 2,
        'n_blue': 3,
        'modifiers': {hit: 1, goblin: 0},
        'effects': [(star, lethal)],  # lethal vs humanoids
    },
    'war_shield': {
        'n_red': 1,
        'n_blue': 2,
        'modifiers': {hit: 1, bash: 1},
        'effects': [],
    },
    'righteous_word': {  # levels up at level 5 (n_red and modifiers)
        'n_red': 0,
        'n_blue': 4,
        'modifiers': {hit: 1},
        'effects': [(lightning, fire), (star, AoE), (lightning+star, critical)],
    },
    'king_of_the_wild': {
        'n_red': 2,
        'n_blue': 3,
        'modifiers': {hit: 1},  # can be hit: 2 for enemy race
        'effects': [(SS, critical)],
    },
    'king_of_the_wild_orcish_arrows': {
        'n_red': 2,
        'n_blue': 3,
        'modifiers': {hit: 1},  # can be hit: 2 for enemy race
        'effects': [(SS, critical), (lightning, hit)],
    },
    'king_of_the_wild_cursed_arrows': {
        'n_red': 2,
        'n_blue': 3,
        'modifiers': {hit: 1},  # can be hit: 2 for enemy race
        'effects': [(SS, critical), (lightning, bleed+bleed)],
    },
    'darts': {
        'n_red': 1,
        'n_blue': 1,
        'modifiers': {},
        'effects': [(lightning, reroll), (star, lethal)],
    },
    'torch': {
        'n_red': 2,
        'n_blue': 0,
        'modifiers': {},
        'effects': [(lightning, fire+fire), (star, AoE)],
    },
    'dharma': {
        'n_red': 2,
        'n_blue': 1,
        'modifiers': {bleed: 1},
        'effects': [(lightning+lightning, attack)],
    },
}

In [5]:
for item, kwargs in weapons.items():
    print()
    print(item)
    display(summarise_simulation(
        simulation(Weapon(**kwargs), n_trials=1000), 
        percentiles=[x/100 for x in range(5, 100, 5)]
))


spear


,mean,std,min,5%,10%,15%,20%,25%,30%,35%,40%,45%,50%,55%,60%,65%,70%,75%,80%,85%,90%,95%,max
💥,4.3,1.2,1,2,3,3,3,4,4,4,4,4,4,4,5,5,5,5,5,6,6,6,7
🩸,0.4,0.5,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1
🧱,0.8,0.4,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1



trident_of_the_deep


,mean,std,min,5%,10%,15%,20%,25%,30%,35%,40%,45%,50%,55%,60%,65%,70%,75%,80%,85%,90%,95%,max
💥,6.5,1.4,2,4,5,5,5,6,6,6,6,6,7,7,7,7,7,7,8,8,8,9,11
💔,0.5,0.5,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1
⛨,0.5,0.5,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1



trident_of_the_deep_1h


...

,mean,std,min,5%,10%,15%,20%,25%,30%,35%,40%,45%,50%,55%,60%,65%,70%,75%,80%,85%,90%,95%,max
💥,1.3,0.8,0,0,0,1,1,1,1,1,1,1,1,1,2,2,2,2,2,2,2,2,3
🎲,0.5,0.5,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1
💀,0.5,0.5,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1



torch


,mean,std,min,5%,10%,15%,20%,25%,30%,35%,40%,45%,50%,55%,60%,65%,70%,75%,80%,85%,90%,95%,max
💥,1.6,0.8,0,0,1,1,1,1,1,1,1,2,2,2,2,2,2,2,2,2,3,3,4
📏,0.3,0.5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1
🔥,1.3,1.0,0,0,0,0,0,0,0,0,2,2,2,2,2,2,2,2,2,2,2,2,2



dharma


,mean,std,min,5%,10%,15%,20%,25%,30%,35%,40%,45%,50%,55%,60%,65%,70%,75%,80%,85%,90%,95%,max
💥,2.2,1.0,0,1,1,1,1,1,2,2,2,2,2,2,2,3,3,3,3,3,3,4,5
🩸,1.0,0.0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
⚔,0.3,0.5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1
